
# TRAP (Trivial Rush Attempt Percentage) with **nflreadpy** (Python)

This notebook reproduces the **TRAP** model popularized by Ben Gretch and demonstrated on Open Source Football — but in Python — using the **nflreadpy** package from the nflverse.

**Definition (working):** TRAP measures the share of a player's **total touches** that are **low-value rush attempts** — specifically, rush attempts **outside the opponent's 10-yard line**. Formally:

\[ \text{TRAP} = \frac{\text{Rush Attempts outside Opp 10}}{\text{Rush Attempts} + \text{Receptions}} \]

> Intuition: touches that *aren't* inside-the-10 rushes or receptions tend to be less valuable for fantasy scoring. A high TRAP suggests a player may be soaking up empty-calorie carries.

---

### What you'll get here
- A simple, configurable pipeline based on `nflreadpy` that loads play-by-play data for your chosen seasons.
- Player-season and multi-season TRAP metrics.
- Plots and tables to quickly spot **TRAP backs** (high TRAP) and **value backs** (low TRAP).
- Clean CSV exports you can plug into league tools.



## Setup

Run the cell below to install dependencies if needed. Then restart the kernel if prompted.


In [ ]:
# If running locally, uncomment to install. Internet is required in your environment to download packages.
# %uv add --upgrade nflreadpy polars pandas matplotlib



## Config

Pick your seasons and any filtering thresholds.


In [ ]:
from datetime import datetime

# --------- User config ---------
SEASONS = list(range(2019, datetime.now().year + 1))  # inclusive; adjust as you like
MIN_TOUCHES = 50  # filter for display/plots; CSVs will include all players regardless

# Plot options
PLOT_CURRENT_SEASON_ONLY = False  # set True to plot just the last season in SEASONS
# --------------------------------

SEASONS


## Imports


In [ ]:
import warnings

warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import polars as pl

try:
    import nflreadpy as nfl
except Exception as e:
    print("If import fails, run the install cell above. Error:", e)


## Load play-by-play data

We pull play-by-play for the selected seasons using `nflreadpy.load_pbp()`.


In [ ]:
# Load play-by-play for selected seasons
pbp = None
try:
    pbp = nfl.load_pbp(seasons=SEASONS)
except Exception as e:
    print(
        "Data load failed. Make sure nflreadpy is installed and you have internet access. Error:", e
    )

pbp.head().to_pandas().head() if pbp is not None else None


## Derive touches and TRAP components

- **Touches** = rush attempts + receptions  
- **Trivial rush attempt**: rush attempt **outside** the opponent's 10 (i.e., `yardline_100 > 10` on rushing plays)


In [ ]:
if pbp is None:
    raise RuntimeError("PBP data not loaded. Aborting.")

# We'll identify rushing plays and receptions.
# nflverse pbp has common fields:
# - rusher_player_id / receiver_player_id
# - rush_attempt (bool/int)
# - pass_attempt (bool/int)
# - complete_pass (bool/int)
# - yardline_100 (yards from end zone; 1 means 1 yard away, 100 at own goal)
# - posteam, season, week, etc.

# Touch flags at the play level
plays = pbp.with_columns(
    [
        # Rush attempt flag
        (pl.col("rush_attempt").fill_null(0) == 1).alias("is_rush"),
        # Reception flag = complete pass credited to a receiver (catch)
        # Many pbp datasets also have "reception" flag; if absent, derive from receiver id & complete_pass
        (
            (pl.col("complete_pass").fill_null(0) == 1) & pl.col("receiver_player_id").is_not_null()
        ).alias("is_rec"),
        # Rush outside opponent 10
        # yardline_100 is distance to end zone for the offense; opp 10 means <=10 yards away.
        ((pl.col("rush_attempt").fill_null(0) == 1) & (pl.col("yardline_100") > 10)).alias(
            "is_trivial_rush"
        ),
    ]
)

# Extract player IDs for rushers and receivers
plays = plays.with_columns(
    [
        pl.when(pl.col("is_rush"))
        .then(pl.col("rusher_player_id"))
        .otherwise(None)
        .alias("rush_player_id"),
        pl.when(pl.col("is_rec"))
        .then(pl.col("receiver_player_id"))
        .otherwise(None)
        .alias("rec_player_id"),
    ]
)

# We'll melt to a "touch-level" table keyed by player
rush_df = (
    plays.filter(pl.col("is_rush"))
    .select(["season", "posteam", "rush_player_id", "is_trivial_rush"])
    .rename({"rush_player_id": "player_id"})
    .with_columns(
        [
            pl.lit(1).alias("rush_att"),
            pl.lit(0).alias("rec"),
        ]
    )
)

rec_df = (
    plays.filter(pl.col("is_rec"))
    .select(["season", "posteam", "rec_player_id"])
    .rename({"rec_player_id": "player_id"})
    .with_columns(
        [
            pl.lit(0).alias("rush_att"),
            pl.lit(1).alias("rec"),
            pl.lit(0).alias("is_trivial_rush"),
        ]
    )
)

# Ensure both have same columns
rush_df = rush_df.with_columns([pl.col("is_trivial_rush").cast(pl.Int64)])
rec_df = rec_df.with_columns([pl.col("is_trivial_rush").cast(pl.Int64)])

touches = pl.concat([rush_df, rec_df], how="diagonal_relaxed")

# Aggregate to player-season
player_season = (
    touches.groupby(["season", "player_id", "posteam"])
    .agg(
        [
            pl.sum("rush_att").alias("rush_att"),
            pl.sum("rec").alias("rec"),
            pl.sum("is_trivial_rush").alias("trivial_rush_att"),
        ]
    )
    .with_columns(
        [
            (pl.col("rush_att") + pl.col("rec")).alias("touches"),
            (pl.col("trivial_rush_att") / (pl.col("rush_att") + pl.col("rec")).clip(lower=1)).alias(
                "trap"
            ),  # avoid div by 0
        ]
    )
)

player_season.head().to_pandas().head()


## Join player names & positions


In [ ]:
players = None
try:
    players = nfl.load_players()
except Exception as e:
    print("Player table load failed. Error:", e)

if players is not None:
    # nflverse players includes 'player_id' (gsis or gsis_it_id) variants; we try several common keys.
    # nflfastR-style pbp usually uses 'player_id' as GSIS ID; nflreadpy should align.
    # We'll try to match on 'gsis_id' if present, else 'nfl_id' or 'pfr_id'. Adjust as needed.
    name_keys = [
        c
        for c in players.columns
        if c in ("gsis_id", "nfl_id", "pfr_id", "esb_id", "gsis_it_id", "player_id")
    ]
    key = "player_id" if "player_id" in name_keys else (name_keys[0] if name_keys else None)

    if key is None:
        display(players.head().to_pandas().head())
        raise RuntimeError(
            "Could not find a compatible player ID column to join on. Inspect players table above."
        )

    players_join = players.select(
        [
            pl.col(key).alias("player_id"),
            pl.col("full_name").alias("player"),
            pl.col("position").alias("pos"),
        ]
    ).unique(subset=["player_id"])

    player_season_named = player_season.join(players_join, on="player_id", how="left")
else:
    player_season_named = player_season.with_columns(
        [pl.lit(None).alias("player"), pl.lit(None).alias("pos")]
    )

player_season_named = player_season_named.select(
    [
        "season",
        "player_id",
        "player",
        "pos",
        "posteam",
        "rush_att",
        "rec",
        "touches",
        "trivial_rush_att",
        "trap",
    ]
)

player_season_named.sort(["season", "trap"], descending=[True, True]).head().to_pandas().head()


## Multi-season aggregates

Roll up to the player level across all selected seasons.


In [ ]:
player_multi = (
    player_season_named.groupby(["player_id", "player", "pos"])
    .agg(
        [
            pl.len().alias("seasons"),
            pl.sum("rush_att").alias("rush_att"),
            pl.sum("rec").alias("rec"),
            pl.sum("touches").alias("touches"),
            pl.sum("trivial_rush_att").alias("trivial_rush_att"),
        ]
    )
    .with_columns([(pl.col("trivial_rush_att") / pl.col("touches").clip(lower=1)).alias("trap")])
)

player_multi.sort("trap", descending=True).head(10).to_pandas()


## Views: leaders & laggards

We show leaders/laggards by TRAP for players with at least `MIN_TOUCHES` over the selected seasons.


In [ ]:
leaders = (
    player_multi.filter(pl.col("touches") >= MIN_TOUCHES)
    .sort(["trap", "touches"], descending=[True, False])
    .with_columns([(pl.col("trap") * 100).round(1).alias("TRAP_%")])
    .select(["player", "pos", "touches", "trivial_rush_att", "TRAP_%"])
    .to_pandas()
)

laggards = (
    player_multi.filter(pl.col("touches") >= MIN_TOUCHES)
    .sort(["trap", "touches"], descending=[False, True])
    .with_columns([(pl.col("trap") * 100).round(1).alias("TRAP_%")])
    .select(["player", "pos", "touches", "trivial_rush_att", "TRAP_%"])
    .to_pandas()
)

import caas_jupyter_tools

caas_jupyter_tools.display_dataframe_to_user("High TRAP (leaders)", leaders)
caas_jupyter_tools.display_dataframe_to_user("Low TRAP (laggards)", laggards)

leaders.head(), laggards.head()


## Plot: TRAP vs. Touches

Each point is a player across all selected seasons (aggregated). High on the y-axis = more trivial carries.


In [ ]:
pm = player_multi.to_pandas()

# Filter for plot readability
plot_df = pm[pm["touches"] >= MIN_TOUCHES].copy()
if PLOT_CURRENT_SEASON_ONLY and len(SEASONS) > 0:
    latest = max(SEASONS)
    # restrict to players active in latest season (approximate: appear in that season table)
    act_latest = set(player_season_named.filter(pl.col("season") == latest)["player_id"].to_list())
    plot_df = plot_df[plot_df["player_id"].isin(act_latest)]

plt.figure(figsize=(8, 6))
plt.scatter(plot_df["touches"], plot_df["trap"] * 100, alpha=0.6)
plt.xlabel("Touches (agg across selected seasons)")
plt.ylabel("TRAP % (Trivial Rush Attempt %)")
plt.title("TRAP vs. Touches")
plt.grid(True)
plt.show()


## Player-season table

Handy if you want to look at TRAP on a per-season basis.


In [ ]:
ps = player_season_named.with_columns([(pl.col("trap") * 100).round(1).alias("TRAP_%")])
ps_out = ps.select(
    [
        "season",
        "player",
        "pos",
        "posteam",
        "rush_att",
        "rec",
        "touches",
        "trivial_rush_att",
        "TRAP_%",
    ]
).to_pandas()

import caas_jupyter_tools

caas_jupyter_tools.display_dataframe_to_user("Player-Season TRAP", ps_out)

ps_out.head()


## Export

Save CSVs for downstream use.


In [ ]:
out_dir = Path("/mnt/data/trap_outputs")
out_dir.mkdir(parents=True, exist_ok=True)

leaders.to_csv(out_dir / "trap_leaders.csv", index=False)
laggards.to_csv(out_dir / "trap_laggards.csv", index=False)
ps_out.to_csv(out_dir / "trap_player_seasons.csv", index=False)

print("Saved:")
for p in sorted(out_dir.glob("*.csv")):
    print(" -", p)


## Notes & caveats

- This follows the **standard** TRAP definition: rushes **outside** the opponent's 10 are considered *trivial*; receptions and rushes **inside** the 10 are *non-trivial*.
- Play-by-play schemas can evolve; if joins on player IDs fail, inspect `players` to pick the right key.
- For finer control, adapt the filters (e.g., exclude kneel-downs, spikes, or certain game situations). Add filters on `qb_kneel`, `qb_spike`, or `play_type` if present.
- Fantasy scoring systems vary; TRAP is **format-agnostic** but aligns most naturally with PPR/half-PPR logic.
